In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# mrjob in the Cloud


[`mrjob`](https://github.com/Yelp/mrjob) is a Python library originally developed by Yelp to streamline the writing and execution of Hadoop Streaming jobs both locally and on the cloud. Instead of directly calling the Hadoop Streaming API, `mrjob` allows us to write Python code and handle much of the cluster configuration automatically.

This notebook has information on using `mrjob` with two of the most popular cloud services, Amazon Web Services (AWS) and Google Cloud Platform (GCP).

## Amazon's Cloud Computing Services


Amazon Web Services has extremely thorough documentation around everything from the commands available to the command line interface (CLI) `aws`,
to the Python wrapper for said interface `boto`, to full tutorials and examples on how to fire up an Elastic MapReduce (EMR) cluster or a bunch of Elastic Compute Cloud (EC2) instances with almost any desired data processing framework.

EC2 is cheaper than EMR, but EMR is recommended for immediate use of Hadoop and any other project in the ecosystem (configurable for your cluster via [Amazon Machine Images](https://docs.aws.amazon.com/emr/latest/ReleaseGuide/emr-release-components.html) (AMIs), also called "software configuration release". In a production setting it's possible you'll want to use specific versions for consistency; in our case it's safe to use the most recent version (`5.17.0` at the time of this writing).

### Setting up a personal AWS account


To use AWS you'll need to [create an account](http://aws.amazon.com/) if you haven't already. For the first year after new account creation, you'll be eligible for discounts on some services as part of the Free Tier program.

You can access the AWS [web console](https://console.aws.amazon.com/s3/) to handle most of your configuration, though you can use the CLI as well. You'll need at least one S3 bucket to serve as storage for your logs and output.

From there you can create EMR clusters as you wish and run jobs. Be careful about the nodes you use, as only certain sizes are eligible for the free tier discounts. Still, you only pay for what you use, and the costs for small, educational jobs are relatively manageable.

There's an in depth [tutorial](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-gs.html) available, and more detailed cluster configuration information can be found in this notebook and in the Spark module.

Once you have an account, you'll need an account key pair for API access (you can't use your username and password with the command line).  To create one, go to Services -> IAM -> Encryption Keys (on the bottom of the left side menu). Click on "Get started" if you don't see a "Create key" option.  

Also note that in addition to the normal credentials, you might need to take care of:
* Generating an EC2 key pair (this is separate from the AWS general key pair, and gives you a `.pem` file to allow SSH access to the EMR machines, instructions [here](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html#having-ec2-create-your-key-pair))
* `aws emr create-default-roles` if you plan on just using the defaults for EMR
* Making sure your user is part of a group with sufficient permissions (admin is probably fine)

Since we are using `mrjob`, we will need to install additional python packages beyond the base EMR image.  We do this through a bootstrap script, a shell script we upload to `S3`.  Create a script named `init.sh` (or, really, anything you want that ends in `.sh`) that contains the following, noting that the standard EMR image has `pip` already installed:

```
sudo pip-3.4 install mrjob
sudo pip-3.4 install simplejson
sudo pip-3.4 install mwparserfromhell
sudo pip-3.4 install lxml
```

Create your EMR cluster by going to Services -> Analytics -> EMR, then clicking on "Create cluster".  At the top there is a "Go to advanced options".  In there, the first two pages of selections are normally fine, but we need to make a change on the third page (General Cluster Settings).  At the top, set the cluster name to whatever you'd like. At the bottom, click on the "Bootstrap actions" tab, and select a "Custom action".  Click "Configure and add", then select your script on the "Script location" and Add it.  On the last page, select your EC2 pair from the drop-down list, then click "Create".

Once this has completed, in the console you will see your new cluster being created.  If you click on the name of the cluster, the information page will come up.  We need two pieces of information: the cluster ID (looks like `j-XXXXXXX`) and the availability zone (looks like `us-east-1b`).  If we strip the last character off of the zone, this is our region, which is what we actually want.

### Managing AWS credentials on our cloud machines


By default, `mrjob` will look in several places for AWS keys (in this order): `~/.mrjob.conf`, the environment variables `AWS_SECRET_ACCESS_KEY` and `AWS_ACCESS_KEY_ID`, and `~/.aws/credentials`.  If you want to use your personal AWS keys for S3 and EMR access _within `mrjob`_, you can create `~/.mrjob.conf`.  The default behavior in `mrjob` is to print the output of the operation to standard out (i.e. the terminal).  Since it (by default) cannot directly query the machines running EMR, it waits for 10 minutes for the output files to make it to S3.  We can get around this either by instructing it not to print the output to the screen by passing the job the flags `--no-output --output-dir=s3://<bucket>/<user>-wc/`, which will instead simply save the results to S3, or by altering our `.mrjob.conf` file to know where to look for the EC2 key pair `.pem` file we generated earlier.  An example file with this and AWS keys set is:

```
runners:
  emr:
    aws_access_key_id: <access key>
    aws_secret_access_key: <secret key>
    ec2_key_pair: EMR
    ec2_key_pair_file: <pem file location>
    ssh_tunnel: true
```

Our AWS keys for our S3 bucket are saved as environment variables.  They have precedent for the `aws` command line client, so you will need to do a little work to use your own S3 buckets this way. You can set up multiple profiles in the `~/.aws/credentials` file in order to facilitate copying data from our S3 bucket while still being able to access your own.  Lots of details are in the [CLI docs](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-getting-started.html).

Once everything is set up, we will need to tell the system to use EMR with our cluster:
```bash
python3 wordcount.py -r emr --region us-east-1 --cluster-id j-1EL1EM3ADMTBS sample.txt
```
of course, replace the `us-east-1` and the `j-1EL1EM3ADMTBS` with your own region and cluster ID, respectively.

In general, if you are running into errors, make sure that the configuration in `~/.mrjob.conf` makes sense. The `mrjob` [docs](https://mrjob.readthedocs.io/en/latest/) and [guides](https://github.com/Yelp/mrjob/tree/master/docs/guides) explain all of the options.


### AWS credentials and command line tools


1. If you have set up the AWS command line interface by setting your key-related environment variables, verify that it is working by trying
``` bash
aws s3 ls
```
You should get back a list of the buckets in your S3.  If you get some sort of access error, check that your environment variables (or `~/.aws/credentials`) is set up correctly.

1. `boto` ([docs](https://boto.readthedocs.org/en/latest/)) and `boto3` ([docs](https://boto3.readthedocs.io/en/latest/)) are Python libraries that wrap the functionality of `awscli`.  You can install them using
``` bash
pip install boto
pip install boto3
```
and follow the instructions in the docs to get started (you don't need both, usually one is sufficient). They are already installed on our cloud machines.

1. Another option for interacting with s3 from the command line is `s3cmd`. You can download/start using it via   
``` bash
git clone https://github.com/s3tools/s3cmd.git
```
and follow the documentation [here](https://github.com/s3tools/s3cmd).


### Python `mrjob`


To test that everything is working correctly, instead of using your own code you can clone this [`github repo`](https://github.com/Yelp/mrjob) and run word count on `README.rst`:

```bash
git clone https://github.com/Yelp/mrjob.git
cd mrjob

# run command locally
# this is good for testing and debugging on files locally

python mrjob/examples/mr_word_freq_count.py README.rst\
   --no-output --output-dir=/tmp/wc/
   
# check the output file contents:

cat /tmp/wc/* | more

# run command on ec2 and write output to our s3 bucket
# this costs money so only do it when you have working code
# Note that this creates a new EMR cluster, and may react poorly
# if you have the ec2_key_pair set in your .mrjob.conf

python mrjob/examples/mr_word_freq_count.py -r emr README.rst \
   --no-output --output-dir=s3://<bucket>/<user>-wc/
```


### check the output file contents:

```bash
aws s3 ls s3://<bucket>/<user>-wc/
aws s3 cp --recursive s3://<bucket>/<user>-wc/ /tmp/<user>-wc/
```
Note: be sure to fill in `<user>` with a key that is unique to you.

A few notes:
1. You can also upload files to s3 using the AWS CLI so that your entire workflow can be on s3.
1. The server will take a while to boot up the first time but it will stay alive.  Any subsequent jobs that are submitted will not have to reboot.  If there are already jobs running, it will wait up to 5 minutes before spawning another server (please be patient).  It will stay idle for 2 hours and then kill itself.
1. Take a look at `mrjob/examples/mr_word_freq_count.py`.  This is the simple "word count" MapReduce.


### Useful AWS CLI commands


- Access Hadoop web UI, e.g. `ResourceManager`
    - Option 1: via a local port
```bash 
ssh -L 8158:ec2-52-0-25-37.compute-1.amazonaws.com:9026 hadoop@ec2-52-0-25-37.compute-1.amazonaws.com -i /path/to/fellos201501.pem
```
Now in your browser, go to the address `localhost:8158`  

    - Option 2: via dynamic port forwarding
        1. Type: 
```bash
aws emr socks --region <region> --cluster-id j-XXXX --key-pair-file ~/path/to/keypair.pem
```

        1. Then follow [these steps](http://docs.aws.amazon.com/ElasticMapReduce/latest/DeveloperGuide/emr-connect-master-node-proxy.html) to add `FoxyProxy` to your Chrome/Firefox browser to natively use the web UI.


- `ssh` into master node to access HDFS, Pig interactive, etc.

```bash
ssh hadoop@{master-public-dns}.compute-1.amazonaws.com -i /path/to/pemfile.pem
```

- One-liner to preview a `.gz` file:

```bash
s3cmd get s3://dataincubator-course/wikidata/wikistats/pagecounts/pagecounts-20081208-030000.gz - | gunzip -c | less
```


### Third party software


There are some third-party tools that can help navigate AWS S3. It can be time consuming to go through the command line looking for logs when there's no autocomplete or easily viewable directory structure - in which case something like Bucket Explorer might save you some time.

## Google Cloud Platform


Cloud Dataproc is GCP's analog to EMR: a managed Hadoop cluster environment that uses Google Compute Engine instances under the hood. There's a comprehensive 60-day free trial with $300 of credit to use (which should be more than plenty for our purposes).  You can interact with the compute engine either through the [Cloud Console website](https://console.cloud.google.com/) or through the `gcloud` command line tool ([install instructions](https://cloud.google.com/sdk/#Quick_Start)).  Here are some step-by-step instructions to get started, largely using the website.

1. Go to the Console website and sign up for a trial account. It will request your credit card information, but you will not be charged.

1. In the top bar of the website, there should be a "Go to projects" drop down.  Open it and select "Create a project...".  You will be prompted for a name, and then allowed to create the project.  The project acts as an umbrella for various related resources.

1. From the menu in the upper left, select "Storage", and then create a bucket.  This is essentially a cloud file system onto which you can load files and directories.  Give it a unique name.

1. From the same menu in the upper left, select "API Manager".  Ensure that the "Compute Engine API", the "Cloud Dataproc API", and the  "Cloud Storage JSON API" are all enabled.

1. Now, switch to the command line on your JupyterHub pod.  Run `gcloud init`.  It will ask you to authenticate with your Google account, as well as set the default project and compute zone.

  1. `gcloud` may not be installed on your pod.  If it is not, since it runs Linux, follow the installation instructions at [Linux Install](https://cloud.google.com/sdk/docs/quickstart-linux).  Note that you'll need to use the option without launching a web browser in the initialization step: `gcloud init --console-only` and you can safely ignore the commentary about Python versions, it will do the right thing.

  1. This should set things up to authenticate all `gcloud` commands.  But if you're having trouble, go to the "Credentials" tab in the "API Manager" section of the website.  Chose "Create Credentials" and choose "Service account key". Then, select New service account, enter a Name and select Key type JSON. Put this JSON file somewhere and point to it with the environment variable $GOOGLE_APPLICATION_CREDENTIALS (eg. `export GOOGLE_APPLICATION_CREDENTIALS=/home/jovyan/auth/gcp.json`). Make sure the service account has editor-level permissions.  There's an option to select "Compute Engine default service account", this usually has everything you need.

1. Upload data to the bucket you created above, using the `gsutil cp` command.  It can take as arguments local paths, Google Storage paths (`gs://bucket-name/path`), and Amazon S3 paths (`s3://bucket-name/path`).  Useful flags include `-r` (recursive) and `-m` (parallelize tasks).  For example, `gsutil -m cp -r s3://dataincubator-course/mrdata/english/ gs://mybucket/data/wikipedia/`.

1. Run a job with `mrjob`.  You can either let `mrjob` create a cluster for you, or you can provision one by hand.  The latter is recommended, as it is easier to configure.

  1. If no cluster is specified, `mrjob` will create a cluster for you.  For example,
    ```bash
      python script.py -r dataproc \
          gs://bucket/directory > out.txt
    ```
  To use third-party libraries you'll need to configure the bootstrapping option in `~/.mrjob.conf`. You can also specify cluster creation options here. For the free trial, remember that not all options are available.
  
  1. You can create a cluster using the Cloud SDK Website.  From the main menu, select "Dataproc", and then choose "Create cluster".  You can configure the cluster, including using SSDs, which often helps performance.  Please note which region and zone the cluster is created in (this is an option, and the results are shown on the list of clusters).
  
      Before you do so, you need to be aware that the cluster will not be provisioned with many libraries.  Critically, `mrjob` itself isn't included.  The easiest solution is to create a provisioning script that looks something like this:
        ```bash
          #!/bin/bash
          apt-get install -y python3-dev
          curl https://bootstrap.pypa.io/get-pip.py | python3
          pip3 install mrjob
          pip3 install simplejson
          pip3 install mwparserfromhell
          pip3 install lxml
        ```
      Upload this script to a Google Storage bucket (`gsutil cp`).  When you are creating a cluster, select the advanced options (at the bottom - "Preemptible workers, bucket, network, version, initialization, & access options") and enter the `gs://` path to the `init` script in the "Initialization actions" entry.  This will cause each node in the cluster to run this script, installing `pip`, `mrjob`, etc.
  
      When the cluster has started running, which takes a few minutes, you can launch your job as before, but now we must specify the cluster and either the region _or_ the zone (not both).  It doesn't matter which, unless your region is "global", in which case you must specify the zone. If you specify neither, it defaults to using `--region us-west1`
      ```bash
          python script.py -r dataproc --zone us-west1-b\
              --cluster-id cluster-1 \
              gs://bucket/directory > out.txt
      ```
      **Remember:** The cluster will not be shut down after your job completes.  Destroy it, from the web console or the command line, to avoid unnecessary charges.

There is a [quick start guide](https://pythonhosted.org/mrjob/guides/dataproc-quickstart.html) to using `mrjob` with Dataproc.  The full [mrjob documentation](https://media.readthedocs.org/pdf/mrjob/latest/mrjob.pdf) contains plenty more details.

### <a name="upgrading"></a>Upgrading from the free trial


The free trial limits you to 8 YARN cores, including the master node, which realistically means the biggest cluster you can use on Dataproc has 3 worker instances with 2 nodes each.

It is a significant time saver to upgrade to a paid account. You will keep your free trial credit (it expires when your trial would have expired), the only difference is **you will have to manually cancel your account** to avoid being billed after the trial period expires. Doing this is painless and will increase your quota to 24 YARN cores.


### Viewing logs


`mrjob` tries to pull back the actual Python errors Tracebacks when a job fails, but as of this writing, this is not implemented for Dataproc.  You can find the logs by SSHing into the worker nodes and examining them directly, but it may be slightly easier to use the log viewer in the web console.  Go to the [Logging page](https://console.cloud.google.com/logs) of the console and select "Dataproc" from the drop-down menu.  You can then select a specific cluster to search, or you can search all clusters.  Searching for "Traceback" will yield all lines that open a Python traceback, but you can't see the whole error message from this view.  Instead, open that line and find the filename, in the `structPayload` structure.  Then search for that filename.  You'll get all the lines of that file, in order, so you can figure out what the error actually was.

Another option: You can open the YARN web interface by following the steps documented by GCP here:
https://cloud.google.com/dataproc/docs/concepts/cluster-web-interfaces
Some logging aggregation is disabled by default on GCP, but you can determine where tasks failed, SSH into worker nodes a little more carefully, and look at logs from there. It's also useful for keeping track of your jobs.

*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*